### 추가 라이브러리 설치됨

In [1]:
import sys
sys.path.append('/workspace')

In [2]:
text1 = "나는 당산에 간다."
text2 = "나는 동싼에 간다."

In [3]:
from app.hangul2ipa.worker import hangul2ipa
import pandas as pd

def parse_ipa(ipa_input):
    ipas = pd.read_csv("./ipa2ko.csv")["IPA"].values

    ipas = sorted(ipas, key=lambda c: len(c), reverse=True)

    parsed_ipa = []
    i = 0

    while i < len(ipa_input):
        skip = True
        for ipa in ipas:
            if ipa == ipa_input[i:i+len(ipa)]:
                skip = False
                parsed_ipa.append(ipa)
                i += len(ipa)
                break
        if skip == True:
            i += 1

    return parsed_ipa

def parse_ipa_with_words(ipa_sentence):
    words = ipa_sentence.split(" ")
    
    parsed_words = list(map(parse_ipa, words))
    
    return parsed_words


In [4]:
from difflib import SequenceMatcher

def compare_ipa_with_word_index(original_ipa, user_ipa):
    parsed_original = parse_ipa_with_words(original_ipa)
    parsed_user = parse_ipa_with_words(user_ipa)
    
    diff_with_word_index = []
    
    # 두 개의 단어 리스트를 단어 단위로 비교
    for word_index, (orig_word, user_word) in enumerate(zip(parsed_original, parsed_user)):
        # SequenceMatcher로 단어 단위의 음소 비교
        matcher = SequenceMatcher(None, orig_word, user_word)
        for tag, i1, i2, j1, j2 in matcher.get_opcodes():
            if tag != 'equal':
                # word_index로 몇 번째 단어에서 차이가 발생했는지 표시
                diff_with_word_index.append((word_index + 1, orig_word[i1:i2][0], user_word[j1:j2][0]))
    
    return diff_with_word_index

In [7]:
ipa1 = hangul2ipa(text1)
ipa2 = hangul2ipa(text2)

a = compare_ipa_with_word_index(ipa1, ipa2)

print(ipa1)
print(ipa2)

print(a)

nɑnɯn tɑŋsɑnɛ kɑndɑ
nɑnɯn toŋs*ɑnɛ kɑndɑ
[(2, 'ɑ', 'o'), (2, 's', 's*')]
